In [1]:
from nlp_api import *
from google.cloud import bigquery
import pandas_gbq

import warnings
warnings.simplefilter("always", category=PendingDeprecationWarning) 
warnings.simplefilter("always", category=DeprecationWarning) 

In [2]:
"""Queries BQ, adds NL API results, and writes back to BQ."""
bq_client = bigquery.Client(location="US", project="ut-goog")
query = """
  SELECT 
    complaint_id, consumer_complaint_narrative
  FROM `bigquery-public-data.cfpb_complaints.complaint_database`
  WHERE consumer_complaint_narrative IS NOT NULL
  LIMIT 20;
"""
dataset = "nl_api"
table = "nl_api_data"
sentiment_path = "./sentiment"

# Load queries into dataframe and load review checkpoints.
df = load_query_results(bq_client, query)

Query results loaded into dataframe!


In [5]:
sentiments = load_checkpoint(sentiment_path)

sentiments = call_nl_api(df, sentiments, sentiment_path,
                       get_sentiment, batch_size=8)

# Add NL API data to dataframes and write to BQ.
df = add_sentiment_data(df, sentiments)

[./sentiment] count: 20
Processed 20 rows in 2.9150032997131348 seconds.
[./sentiment] final count: 20


In [6]:
df

,consumer_complaint_narrative,sentiment_score,sentiment_magnitude
complaint_id,,,
1932807,Received {$50.00} prepaid card AMX when bought...,-0.1,4.4
1604786,Rush card was under maintance Monday XXXX XXXX...,-0.5,1.0
2316550,I receive monthly stipend for being a XXXX via...,-0.5,2.1
2224405,I received a prepaid debit card from Citi bank...,-0.5,1.7
1633424,There 's multiple issues above that affected m...,-0.5,2.0
1566056,"My account, as well as at least two others bel...",-0.7,2.2
2069738,I received a {$60.00} prepaid card as a rebate...,-0.3,1.4
2264861,I have a {$100.00} Visa gift card I was saving...,0.0,3.3
1618486,I have been forced to remain a customer of Rus...,-0.3,6.0


### Write dataframe to BQ Table

In [4]:
project_id = "ut-goog"
table_id = "nl_api.data"

In [5]:
pandas_gbq.to_gbq(df, table_id, project_id=project_id,chunksize=None,
                 if_exists='append',
                 verbose=False)

/home/jupyter/.local/lib/python3.5/site-packages/pandas_gbq/gbq.py:1152: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
/home/jupyter/.local/lib/python3.5/site-packages/pandas_gbq/gbq.py:1297: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  table_ref = self.client.dataset(self.dataset_id).table(table_id)
/home/jupyter/.local/lib/python3.5/site-packages/pandas_gbq/gbq.py:1297: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  table_ref = self.client.dataset(self.dataset_id).table(table_id)
/home/jupyter/.local/lib/python3.5/site-packages/pandas_gbq/gbq.py:1404: PendingDeprecationWarning: Client.d

In [4]:
sentiments = load_checkpoint(sentiment_path)

sentiments = call_nl_api(df, sentiments, sentiment_path,
                       get_sentiment, batch_size=10000)

# Add NL API data to dataframes and write to BQ.
df = add_sentiment_data(df, sentiments)
#write_bq_table(bq_client, df, dataset, table)


[./sentiment] Loaded 0 reviews from checkpoint!
[./sentiment] count: 20
Processed 20 rows in 0.6504917144775391 seconds.
[./sentiment] final count: 20


NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/ut-goog/jobs?uploadType=resumable: Not found: Dataset ut-goog:nl_api

In [5]:
df

,consumer_complaint_narrative,sentiment_score,sentiment_magnitude
complaint_id,,,
1732400,American Express Gift Cards charged {$60.00} i...,-0.1,4.1
1642774,We have XXXX prepaid debit cards from Suntrust...,-0.5,2.9
2414907,I received a {$50.00} American Express Prepaid...,-0.2,4.3
1581811,I enrolled in American Express has premier shi...,-0.3,4.0
2168676,I received a Citi Visa Debit card with a balan...,-0.1,0.7
1326336,I had a prepaid XXXX card issued to me as ince...,-0.2,3.0
1581265,I received XXXX ( XXXX ) USBank Cash+ Bonus Re...,0.0,3.4
2162217,I received a prepaid {$26.00} XXXX Citibank XX...,-0.5,2.8
2084163,After successfully using rebates from merchant...,-0.4,8.9


In [6]:
dataset_name = "nl_api"
table_name = "nl_api.data"

In [7]:
df.index = df.index.astype(int)
dataset = bigquery.dataset.DatasetReference("ut-goog", dataset_name)
#table_ref = dataset.table(table_name)

In [43]:
table_ref

TableReference(DatasetReference('ut-goog', 'nl_api'), 'nl_api.data')

In [ ]:
bigquery.

In [40]:
df

,consumer_complaint_narrative
complaint_id,
1932807,Received {$50.00} prepaid card AMX when bought...
1604786,Rush card was under maintance Monday XXXX XXXX...
2316550,I receive monthly stipend for being a XXXX via...
2224405,I received a prepaid debit card from Citi bank...
1633424,There 's multiple issues above that affected m...
1566056,"My account, as well as at least two others bel..."
2069738,I received a {$60.00} prepaid card as a rebate...
2264861,I have a {$100.00} Visa gift card I was saving...
1618486,I have been forced to remain a customer of Rus...


In [18]:
!pip install pandas_gbq

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
dataframe.to_gbq('my_dataset.my_table', 
                 'my_project_id',
                 chunksize=None, # i've tryed with several chunksizes, it runs faster when is one big chunk (at least for me)
                 if_exists='append',
                 verbose=False
                 )

In [8]:
job_config = bigquery.LoadJobConfig()
job_config.schema = [
bigquery.SchemaField("consumer_complaint_narrative", bigquery.enums.SqlTypeNames.STRING)
    ]
job_config.write_disposition = "WRITE_TRUNCATE"

In [9]:
job = bq_client.load_table_from_dataframe(df, table_name, project = 'ut-goog',location="US",job_config=job_config)
job.result()

In [10]:
assert job.state == "DONE"
print("BQ table updated:", table_name)

BQ table updated: nl_api.data


In [15]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField(name="consumer_complaint_narrative", field_type="STRING"),
        bigquery.SchemaField(name="complaint_id", field_type="INT64")
    ]
)
job_config.write_disposition = "WRITE_TRUNCATE"

In [16]:
job = bq_client.load_table_from_dataframe(df, table_ref, location="US",job_config=job_config)
job.result()

NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/ut-goog/jobs?uploadType=resumable: Not found: Dataset ut-goog:nl_api